In [323]:
# Load Libraries
import pandas as pd
import os
import datetime
import json

In [324]:
from gremlin_python.structure.graph import Graph
from gremlin_python.process.traversal import T
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport

In [332]:
def load_vertex(fdataset_name, fdata_bucket_name):
    print("loading vertex " + fdataset_name)
    
    # Load data from json
    data_location = '{}/{}'.format(fdata_bucket_name, fdataset_name)
    unique_property = 'id'
    vertex ={}
    
    # Load json to datafrase
    vertex_data = {}
    with open(data_location, 'r') as jf:
        vertex_data = json.load(jf)

    # Load to graph
    for data in vertex_data:
        vertex_id = data['id']
        label = data['label']
        properties = data['properties']
        name = data['id']
        
        try:
            existing_vertex = g.V().has(unique_property,properties[unique_property]).next()
            vertex = g.V(existing_vertex)
            print(f"Updated vertex with {unique_property}={vertex_id}")
        except Exception as error:
            vertex = g.addV(label)
            print(f"Insert vertex with {unique_property}={vertex_id}")
        
        #Insert with PK
        #vertex = g.addV(label).property(T.id, vertex_id)            
        # Insert without PK (duplicate data)
        #vertex = g.addV(label).property('id', vertex_id)
    
        # Add properties to the vertex
        for key, value in properties.items():
            vertex = vertex.property(key, value)
        # Execute the query
        try:
            vertex.next()
        except Exception as error:
          print("Vertex error => An exception occurred:", error)

In [333]:
def load_edge(fdataset_name, fdata_bucket_name):
    print("loading edge " + fdataset_name)
    
    # Load data from json
    data_location = '{}/{}'.format(fdata_bucket_name, fdataset_name)

    # Load json to datafrase
    edge_data = {}
    with open(data_location, 'r') as jf:
        edge_data = json.load(jf)

    #print(edge_data)
    
    # Load to graph
    for data in edge_data:
        from_id = data['from_id']
        to_id = data['to_id']
        edge_label = data['label']
        edge_properties = data['properties']

        # Create the edge with the given label and properties
        edge = g.V(from_id).as_('a').V(to_id).addE(edge_label).from_('a')

        # Add properties to the edge
        for key, value in edge_properties.items():
            edge = edge.property(key, value)
        
        # Execute the query
        try:
            edge.next()
        except Exception as error:
          print("Edge error => An exception occurred:", error , "|" , edge )

In [334]:
# Prepare connection
endpoint = 'ws://127.0.0.1:8182/gremlin'
connection = DriverRemoteConnection(endpoint,'g',
                 transport_factory=lambda:AiohttpTransport(call_from_event_loop=True))

graph = Graph()
g = graph.traversal().withRemote(connection)

In [335]:
#from gremlin_python.driver import client, serializer
#endpoint = 'ws://127.0.0.1:8182/gremlin'

#gremlin_client = client.Client(
#    endpoint, 
#    'g',
#    transport_factory=lambda:AiohttpTransport(call_from_event_loop=True),
#    username="",
#    password="",
#    message_serializer=serializer.GraphSONSerializersV2d0()
#)

In [336]:
def upsert_vertex(fdataset_name, fdata_bucket_name):
    print("loading upsert_vertex " + fdataset_name)
    
    # Load data from json
    data_location = '{}/{}'.format(fdata_bucket_name, fdataset_name)

    # Load json to datafrase
    vertex_data = {}
    with open(data_location, 'r') as jf:
        vertex_data = json.load(jf)

    #print(vertex_data)
    
    unique_property = 'id'
    properties = {}
    vertex_label ={}
    
    # Load to graph
    for data in vertex_data:
        vertex_label  = data['label']
        properties = data['properties']
        
        query = f"g.V().has({unique_property},{properties[unique_property]}).fold().coalesce(unfold(),addV('{vertex_label}').property({unique_property},{properties[unique_property]}))"
        #query = f"g.V({properties[unique_property]}).fold().coalesce(unfold(),addV('{vertex_label}').property({unique_property},{properties[unique_property]}))"
    
        for key, value in properties.items():
            if key != unique_property:
                query += f".property('{key}','{value}')"
        
        print(query)
        # Execute the query
        try:
            result = gremlin_client.submit(query).all().result()
        except Exception as error:
          print("Vertex error => An exception occurred:", error)


# Load VERTEX

In [337]:
data_bucket_name = "/mnt/c/Eliezer/workspace/github.com/jupyter-notebook/graph/dataset"

#dataset_name = 'person_vertex.json'
#load_vertex(dataset_name, data_bucket_name)

#dataset_name = 'account_vertex.json'
#load_vertex(dataset_name, data_bucket_name)

#dataset_name = 'card_vertex.json'
#load_vertex(dataset_name, data_bucket_name)

#dataset_name = 'payment_vertex.json'
#load_vertex(dataset_name, data_bucket_name)

dataset_name = 'terminal_vertex.json'
load_vertex(dataset_name, data_bucket_name)

#dataset_name = 'merchant_vertex.json'
#load_vertex(dataset_name, data_bucket_name)

loading vertex terminal_vertex.json
Updated vertex with id=1
Updated vertex with id=2
Updated vertex with id=3
Updated vertex with id=4
Updated vertex with id=5
Updated vertex with id=6
Updated vertex with id=7
Updated vertex with id=8
Updated vertex with id=9
Updated vertex with id=10
Updated vertex with id=11
Updated vertex with id=12
Updated vertex with id=13
Updated vertex with id=14
Updated vertex with id=15
Updated vertex with id=16
Updated vertex with id=17
Updated vertex with id=18
Updated vertex with id=19
Updated vertex with id=20
Updated vertex with id=21
Updated vertex with id=22
Updated vertex with id=23
Updated vertex with id=24
Updated vertex with id=25
Updated vertex with id=26
Updated vertex with id=27
Updated vertex with id=28
Updated vertex with id=29
Updated vertex with id=30
Updated vertex with id=31
Updated vertex with id=32
Updated vertex with id=33
Updated vertex with id=34
Updated vertex with id=35
Updated vertex with id=36
Updated vertex with id=37
Updated ver

# Load Edge

In [18]:
dataset_name = 'person-account-edge-v2.json'
load_edge(dataset_name, data_bucket_name)

dataset_name = 'account-card-edge.json'
load_edge(dataset_name, data_bucket_name)

dataset_name = 'card-payment-edge.json'
load_edge(dataset_name, data_bucket_name)

dataset_name = 'payment-mcc-edge.json'
load_edge(dataset_name, data_bucket_name)

dataset_name = 'payment-terminal-edge.json'
load_edge(dataset_name, data_bucket_name)

loading edge person-account-edge-v2.json
Edge error => An exception occurred:  | [['V', 509], ['as', 'a'], ['V', 10001], ['addE', 'has'], ['from', 'a'], ['property', 'id', '509-10001-ACC-PER']]
Edge error => An exception occurred:  | [['V', 1165], ['as', 'a'], ['V', 10002], ['addE', 'has'], ['from', 'a'], ['property', 'id', '1165-10002-ACC-PER']]
Edge error => An exception occurred:  | [['V', 1689], ['as', 'a'], ['V', 10003], ['addE', 'has'], ['from', 'a'], ['property', 'id', '1689-10003-ACC-PER']]
Edge error => An exception occurred:  | [['V', 2354], ['as', 'a'], ['V', 10004], ['addE', 'has'], ['from', 'a'], ['property', 'id', '2354-10004-ACC-PER']]
Edge error => An exception occurred:  | [['V', 299], ['as', 'a'], ['V', 10005], ['addE', 'has'], ['from', 'a'], ['property', 'id', '299-10005-ACC-PER']]
Edge error => An exception occurred:  | [['V', 2492], ['as', 'a'], ['V', 10006], ['addE', 'has'], ['from', 'a'], ['property', 'id', '2492-10006-ACC-PER']]
Edge error => An exception occurr

In [ ]:
connection.close()

# Query

In [ ]:
#for i in range(len(data_person)):
    
#    vertex=data_person.iloc[i, 2]
#    id=data_person.iloc[i, 1]
#    name=data_person.iloc[i, 3]
    
#    print('data : ',id, vertex, name)
    
#    person = g.V(vertex).property('name',name).as_(id)
#person.next()

In [ ]:
#x = g.V('P-2354').valueMap().next()
#print(x)